In [ ]:
""" 제 1고지 미분 자동 계산 """

In [ ]:
""" STEP9. 함수를 더 편리하게 """


import numpy as np

# ndarray 인스턴스만 취급하고록 바꿈 ( 다른게 들어오면 오류 )

# Variable이라는 상자 생성
class Variable:
    def __init__(self, data): # 생성자
        # 입력 데이터가 None이 아닌 경우, 
        # 입력 데이터의 타입이 np.ndarray인지 확인
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(
                    '{}은(는) 지원하지 않습니다.'.format(type(data)))
         
        self.data = data # 변수의 데이터를 입력 데이터로 설정

        # 변수의 기울기 초기화
        self.grad = None # 미분값 저장하기 위한 변수

        # 변수를 생성한 함수(연산) 초기화
        self.creator = None # 연산을 나타내는 객체

    # 해당 변수가 어떤 함수에 의해 만들어졌는지를 저장
    def set_creator(self, func):
        self.creator = func

    # 역전파를 자동화 할 수 있도록 새로운 메서드 생성
    # Step7과 다른점 : 처리 효율을 개선하고 앞으로 확장을 대비
    """ # 재귀를 사용한 구현
        def backward(self):
        f = self.creator # 함수를 가져온다.
        if f is not None:
            x = f.input # 2. 함수의 입력을 가져온다.

            # 함수의 backward 메서드를 호출한다.
            x.grad = f.backward(self.grad)

            # 하나 앞 변수의 backward 메서드를 호출한다. ( 재귀 )
            x.backward() """

    # 반복문을 이용한 구현
    def backward(self):
        # y.grad = np.array(1.0) 생략을 위한 if문
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]# 함수를 리스트에 저장
        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            x, y = f.input, f.output # 함수의 입출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                # 하나 앞의 함수를 리스트에 추가한다.
                funcs.append(x.creator)

In [ ]:
# 주어진 입력을 NumPy 배열로 변환하는 함수
def as_array(x):
    if np.isscalar(x):  # 입력이 스칼라인지 확인
        return np.array(x)  # 스칼라인 경우, 배열로 변환하여 반환
    return x  # 스칼라가 아닌 경우, 그대로 반환

In [ ]:
# Variable 인스천스를 변수로 다룰 수 있는 함수를 Function클래스로 구현
class Function:
    def __call__(self, input):
        x = input.data # 데이터를 꺼냄
        y = self.forward(x) # 구체적인 계산 == forward 메서드에서 함

        # Variable 형태로 되돌림
        output = Variable(as_array(y)) # output은 항상 ndarray가 됨 !
        output.set_creator(self) # 출력 변수에 창조자 설정
        self.input = input # 입력 저장
        self.output = output # 출력 저장
        return output
    
    # 순전파
    def forward(self, x):
        raise NotImplementedError()
    
    # 역전파
    def backward(self, gy):
        raise NotImplementedError()

In [ ]:
# y = x²
class Square(Function):
    # 순전파
    def forward(self, x):
        y = x ** 2 # y = x²
        return y
    
    # 역전파
    def backward(self, gy): # gy = 출력쪽에 전해지는 미분값을 전달하는 역할
        x = self.input.data
        gx = 2 * x * gy #  y' = 2x
        return gx

In [ ]:
""" STEP10. 테스트 """


import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = Square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)